In [1]:
# mlp for multi-label classification
import numpy as np
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
# get the dataset
def get_dataset():
 df = pd.read_csv('E:\\Praca_magisterska\\narrative_system\\data\\creating_data\\random_states_nn.csv')
 df.drop(['Unnamed: 0','results','index'],axis=1,inplace=True)
 
 
 

 y= df[['survives', 'helps_all' , 'helps_ego']]
 
 X = df.drop(['survives', 'helps_all' , 'helps_ego'], axis=1)
 print(X.columns)
 print(y.columns)
 categorical_feature_mask = X.dtypes==object
# filter categorical columns using mask and turn it into a list
 categorical_cols = X.columns[categorical_feature_mask].tolist()

 for col in categorical_cols:
  one_hot = pd.get_dummies(X[col])
  X = X.drop(col,axis = 1)
# Join the encoded df
  X = X.join(one_hot)

 

 return X.to_numpy(), y.to_numpy()


In [6]:

# define dataset
X, y = get_dataset()
# summarize dataset shape
print(X.shape, y.shape)
# summarize first few examples
for i in range(1):
 print(X[i], y[i])

Index(['char_acting_altruism', 'char_acting_ambition',
       'char_acting_fighting_ability', 'char_acting_safety',
       'char_acting_food', 'char_acting_satisfaction',
       'char_acting_likes_char1', 'char_acting_likes_char2', 'char1_altruism',
       'char1_ambition', 'char1_fighting_ability', 'char1_safety',
       'char1_food', 'char1_satisfaction', 'char1_likes_char_acting',
       'char1_likes_char2', 'char2_altruism', 'char2_ambition',
       'char2_fighting_ability', 'char2_safety', 'char2_food',
       'char2_satisfaction', 'char2_likes_char_acting', 'char2_likes_char1',
       'ghost_relations', 'overall_food', 'overall_coins'],
      dtype='object')
Index(['survives', 'helps_all', 'helps_ego'], dtype='object')
(4522, 27) (4522, 3)
[-1  0  0 -1  0 -1  0  1  1  1 -1  2  0  1 -1  1 -1  1 -1  0  0  1  0  1
  0  1  1] [1 0 0]


In [7]:

# get the model
def get_model(n_inputs, n_outputs):
 model = Sequential()
 model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(16,  kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(n_outputs, activation='sigmoid'))
 model.compile(loss='binary_crossentropy', optimizer='adam')
 
 return model

In [8]:

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 # enumerate folds
 for train_ix, test_ix in cv.split(X):
    # prepare data
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, verbose=0, epochs=100,batch_size=15)
    # make a prediction on the test set
    yhat = model.predict(X_test)
    #print(np.argmax(model.predict(X_test), axis=-1))
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    # store result
    print('>%.3f' % acc)
    results.append(acc)
 return results

In [9]:
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

Index(['char_acting_altruism', 'char_acting_ambition',
       'char_acting_fighting_ability', 'char_acting_safety',
       'char_acting_food', 'char_acting_satisfaction',
       'char_acting_likes_char1', 'char_acting_likes_char2', 'char1_altruism',
       'char1_ambition', 'char1_fighting_ability', 'char1_safety',
       'char1_food', 'char1_satisfaction', 'char1_likes_char_acting',
       'char1_likes_char2', 'char2_altruism', 'char2_ambition',
       'char2_fighting_ability', 'char2_safety', 'char2_food',
       'char2_satisfaction', 'char2_likes_char_acting', 'char2_likes_char1',
       'ghost_relations', 'overall_food', 'overall_coins'],
      dtype='object')
Index(['survives', 'helps_all', 'helps_ego'], dtype='object')
15/15 [==============================] - 0s 1ms/step
>0.987
15/15 [==============================] - 0s 2ms/step
>0.989
15/15 [==============================] - 0s 2ms/step
>0.993
15/15 [==============================] - 0s 2ms/step
>0.998
15/15 [==================

In [10]:
from numpy import asarray
n_inputs, n_outputs = X.shape[1], y.shape[1]
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
model.save('model1.keras')
# make a prediction for new data


['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active']
['wants_meds', 'wants_food', 'attacks_doctor', 'attacks_soldier',
       'attacks_actor']

In [11]:
np.set_printoptions(suppress=True)
row = [-1,  0 , 0, -1,  0, -1,  0,  1,  1 , 1 ,-1,  2,  0,  1, -1 , 1, -1,  1 ,-1,  0,  0, 1 , 0 , 1,
  0,  1 , 1]
newX = asarray([row])
yhat = model.predict(newX)
print(np.argmax(yhat, axis=-1))
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 89ms/step
[0]
Predicted: [0.99975103 0.         0.        ]


: 